In [28]:
clear all

In [29]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io 
import numpy as np
import pandas as pd
from scipy.signal import butter, lfilter
from scipy.fft import fft
import random
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier



In [30]:
faultI10=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_1_10hz.mat')
faultI30=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_1_30hz.mat')
faultII10=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_2_10hz.mat')
faultII30=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_2_30hz.mat')
faultIII10=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_3_10hz.mat')
faultIII30=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_3_30hz.mat')
faultIV10=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_4_10hz.mat')
faultIV30=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_4_30hz.mat')
faultV10=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_5_10hz.mat')
faultV30=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_5_30hz.mat')





In [31]:
sigII30=faultII30['Acquisition_un_2_30hz']
sigII10=faultII10['Acquisition_un_2_10hz']
del faultII30,faultII10
sigI30=faultI30['Acquisition_un_1_30hz']
sigI10=faultI10['Acquisition_un_1_10hz']
del faultI30,faultI10
sigIII10=faultIII10['Acquisition_un_3_10hz']
sigIII30=faultIII30['Acquisition_un_3_30hz']
del faultIII10,faultIII30
sigIV10=faultIV10['Acquisition_un_4_10hz']
sigIV30=faultIV30['Acquisition_un_4_30hz']
del faultIV10,faultIV30
sigV10=faultV10['Acquisition_un_5_10hz']
sigV30=faultV30['Acquisition_un_5_30hz']
del faultV10,faultV30
# sigVI10=faultVI10['Acquisition_un_6_10hz']
# sigVI30=faultVI30['Acquisition_un_6_30hz']
# del faultVI10,faultVI30

In [32]:

def sliding_window(data, window_size, stride):
    """
    Applies a sliding window to the given data with the specified window size
    and stride length. Padding is added at the beginning and end of the data to
    ensure all samples are included in windows.
    """
    num_padding = window_size - stride
    data_padded = np.pad(data, ((num_padding, num_padding), (0, 0)), 'constant')
    window_data = []
    for i in range(0, len(data_padded) - window_size + 1, stride):
        window = data_padded[i:i+window_size, :]
        window_data.append(window)
    window_data = np.array(window_data)
    # window_data = window_data[:training_size, :, :]
    window_data = np.transpose(window_data, (0, 2, 1))
    return window_data

In [33]:
windowsize=1500
stride=1400

In [34]:
sigtI10hz=sliding_window(sigI10,windowsize,stride)[:,1:13,:]
sigtI30hz=sliding_window(sigI30,windowsize,stride)[:,1:13,:]
sigtII10hz=sliding_window(sigII10,windowsize,stride)[:,1:13,:]
sigtII30hz=sliding_window(sigII30,windowsize,stride)[:,1:13,:]
del sigI10,sigI30,sigII10,sigII30
sigtIII30hz=sliding_window(sigIII30,windowsize,stride)[:,1:13,:]
sigtIII10hz=sliding_window(sigIII10,windowsize,stride)[:,1:13,:]
sigtIV30hz=sliding_window(sigIV30,windowsize,stride)[:,1:13,:]
sigtIV10hz=sliding_window(sigIV10,windowsize,stride)[:,1:13,:]
del sigIII30,sigIII10,sigIV30,sigIV10
sigtV30hz=sliding_window(sigV30,windowsize,stride)[:,1:13,:]
sigtV10hz=sliding_window(sigV10,windowsize,stride)[:,1:13,:]
# sigtVI30hz=sliding_window(sigVI30,windowsize,stride)
# sigtVI10hz=sliding_window(sigVI10,windowsize,stride)
# del sigV30,sigV10,sigVI30,sigVI10

In [35]:
datalable1=np.concatenate([sigtI10hz,sigtI30hz],axis=-1)

datalable2=np.concatenate([sigtII10hz,sigtII30hz],axis=-1)

datalable3=np.concatenate([sigtIII10hz,sigtIII30hz],axis=-1)

datalable4=np.concatenate([sigtIV10hz,sigtIV30hz],axis=-1)

datalabele5=np.concatenate([sigtV10hz,sigtV30hz],axis=-1)


In [36]:
#labels=np.concatenate((np.ones(np.shape(datalable1)[0]),np.zeros(np.shape(datalable1)[0]),np.ones(np.shape(datalable1)[0])*4,np.ones(np.shape(datalable1)[0])*2,np.ones(np.shape(datalable1)[0])*3),axis=0)
dataall=np.concatenate([datalable1,datalable2,datalable3,datalable4,datalabele5],axis=0)

In [37]:
np.shape(dataall)

(2150, 12, 3000)

In [38]:
import numpy as np
from scipy.signal import stft

# generate example signal with shape (7530, 12, 1000)
data = dataall

# set STFT parameters
window = 'hann'
nperseg = 256
noverlap = 128
fs = 10000

# compute STFT spectrogram for each channel
spectrograms = []
for i in range(data.shape[1]):
    _, _, spec = stft(data[:, i, :], fs=fs, window=window, nperseg=nperseg, noverlap=noverlap)
    spectrograms.append(np.abs(spec))

# stack spectrograms along the channel axis to get shape (7530, 12, num_freq_bins, num_time_bins)
spectrograms = np.stack(spectrograms, axis=1)





In [39]:
np.shape(spectrograms)

(2150, 12, 129, 25)

In [40]:
mean = np.mean(spectrograms, axis=(0, 1, 2, 3))
std = np.std(spectrograms, axis=(0, 1, 2, 3))
normalized_data = (spectrograms - mean) / std
data=normalized_data


In [41]:
np.shape(data)

(2150, 12, 129, 25)

In [42]:
labels=np.concatenate((np.ones(np.shape(datalable1)[0]),np.zeros(np.shape(datalable1)[0]),np.ones(np.shape(datalable1)[0])*4,np.ones(np.shape(datalable1)[0])*2,np.ones(np.shape(datalable1)[0])*3),axis=0)

In [43]:
labels

array([1., 1., 1., ..., 3., 3., 3.])

In [44]:
np.size(labels)

2150

In [45]:
indices = np.random.permutation(data.shape[0])

# Use the shuffled indices to shuffle both data and labels
shuffled_data = data[indices]
shuffled_labels = labels[indices]
data= shuffled_data
labels=shuffled_labels


In [46]:
labels

array([2., 4., 0., ..., 4., 3., 0.])

In [47]:
from sklearn.model_selection import train_test_split

# Assume stft_data is your STFT results stored in a variable
# with shape (7530, 12, 129, 9)

# Generate labels for your data (assuming you have labels available)


# Split the data and labels into training and test sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print(f"x_train shape: {x_train.shape}")  # should be (6024, 12, 129, 9)
print(f"y_train shape: {y_train.shape}")  # should be (6024,)
print(f"x_test shape: {x_test.shape}")    # should be (1506, 12, 129, 9)
print(f"y_test shape: {y_test.shape}")    # should be (1506,)


x_train shape: (1720, 12, 129, 25)
y_train shape: (1720,)
x_test shape: (430, 12, 129, 25)
y_test shape: (430,)


In [48]:
from keras.utils import to_categorical
y_train1 = to_categorical(y_train)

y_test1 = to_categorical(y_test)

In [49]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, concatenate
from tensorflow.keras.models import Model

input_shape = (12, 129, 25)
num_classes = 5
# Define input layer
inputs = Input(shape=input_shape)

# First scale
conv1 = Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same')(inputs)
pool1 = MaxPooling2D(pool_size=(2,2))(conv1)

# Second scale
conv2 = Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same')(pool1)
pool2 = MaxPooling2D(pool_size=(2,2))(conv2)

# Third scale
conv3 = Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same')(pool2)
pool3 = MaxPooling2D(pool_size=(2,2))(conv3)

# Flatten and concatenate scales
flatten = Flatten()(pool3)
dense1 = Dense(128, activation='relu')(flatten)
dropout1 = Dropout(0.5)(dense1)

# Add final output layer
outputs = Dense(num_classes, activation='softmax')(dropout1)

# Create model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train1, validation_split=0.1, epochs=40, batch_size=256)


Epoch 1/40
7/7 [==============================] - 4s 499ms/step - loss: 1.8008 - accuracy: 0.2119 - val_loss: 1.6173 - val_accuracy: 0.1977
Epoch 2/40
7/7 [==============================] - 3s 434ms/step - loss: 1.6257 - accuracy: 0.2074 - val_loss: 1.6119 - val_accuracy: 0.1919
Epoch 3/40
7/7 [==============================] - 3s 444ms/step - loss: 1.6227 - accuracy: 0.1938 - val_loss: 1.6102 - val_accuracy: 0.1919
Epoch 4/40
7/7 [==============================] - 4s 518ms/step - loss: 1.6134 - accuracy: 0.2048 - val_loss: 1.6087 - val_accuracy: 0.1919
Epoch 5/40
7/7 [==============================] - 3s 433ms/step - loss: 1.6114 - accuracy: 0.2106 - val_loss: 1.6075 - val_accuracy: 0.2093
Epoch 6/40
7/7 [==============================] - 3s 431ms/step - loss: 1.6118 - accuracy: 0.2145 - val_loss: 1.6052 - val_accuracy: 0.1977
Epoch 7/40
7/7 [==============================] - 3s 432ms/step - loss: 1.6074 - accuracy: 0.2106 - val_loss: 1.6031 - val_accuracy: 0.2151
Epoch 8/40
7/7 [====

: 

: 

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

# Evaluate the model on the test set with multi-class labels
test_loss_multi, test_acc_multi = model.evaluate(x_test, y_test1, verbose=2)
print('Test accuracy multi:', test_acc_multi)

# Convert y_test1 to a multi-class format
y_pred_multi = model.predict(x_test)
y_test_pred_multi = np.argmax(y_pred_multi, axis=1)

# Compute the confusion matrix for multi-class
print('Multi-class:', confusion_matrix(np.argmax(y_test1, axis=1), y_test_pred_multi))

# Evaluate the model on the test set with normal labels
test_loss_normal, test_acc_normal = model.evaluate(x_test, y_test, verbose=2)
print('Test accuracy normal:', test_acc_normal)

# Convert y_test to a multi-class format
y_pred_normal = model.predict(x_test)
y_test_pred_normal = np.argmax(y_pred_normal, axis=1)

# Compute the confusion matrix for normal labels
print('Normal labels:', confusion_matrix(y_test, y_test_pred_normal))


In [ ]:
y_test

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)


In [ ]:
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))


In [ ]:
from sklearn.model_selection import train_test_split

# Assume data is your STFT results stored in a variable
# with shape (7530, 12, 129, 9)

# Reshape the data to have a 1D input shape
data_1d = data.reshape(data.shape[0], -1)

# Generate labels for your data
labels = ...

# Split the data and labels into training and test sets
x_train, x_test, y_train, y_test = train_test_split(data_1d, labels, test_size=0.2, random_state=42)


In [ ]:
np.shape(data_1d)

In [ ]:
np.shape(x_train)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# Define the model architecture
model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=(12, 1161)))
model.add(MaxPooling1D(2))
model.add(Conv1D(32, 3, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
import numpy as np



# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))


In [ ]:
import numpy as np

# Assuming you have 3 classes (0, 1, 2, 3, 4)
num_classes = 5

# Generate random labels for your STFT data
labelss = np.random.randint(0, num_classes, size=(7530,))
labelss

In [ ]:
import numpy as np

# Assuming you have 3 classes (0, 1, 2, 3, 4)
num_classes = 5

# Generate random labels for your STFT data
labelss = np.random.randint(0, num_classes, size=(7530,))
labelss

In [ ]:
import numpy as np

# Assuming you have 3 classes (0, 1, 2, 3, 4)
num_classes = 5

# Generate random labels for your STFT data
labelss = np.random.randint(0, num_classes, size=(7530,))
labelss

In [ ]:
import matplotlib.pyplot as plt

# Select a trial to visualize
trial_num = 10

# Get the STFT spectrograms for all channels of the selected trial
trial_stft = spectrograms[trial_num]

# Set up the plot grid
num_channels = trial_stft.shape[0]
num_cols = 4
num_rows = (num_channels + num_cols - 1) // num_cols
fig, axs = plt.subplots(num_rows, num_cols, figsize=(16, num_rows*4))

# Plot each spectrogram
for i, ax in enumerate(axs.flatten()):
    if i < num_channels:
        spec = trial_stft[i]
        img = ax.pcolormesh(spec, cmap='jet')
        ax.set_title(f'Channel {i+1}')
        ax.set_xlabel('Time')
        ax.set_ylabel('Frequency')
        fig.colorbar(img, ax=ax)

# Set overall title and spacing
fig.suptitle(f'STFT Spectrograms for Trial {trial_num+1}')
fig.tight_layout()
plt.show()




In [ ]:
import numpy as np

# Assuming you have 3 classes (0, 1, 2, 3, 4)
num_classes = 5

# Generate random labels for your STFT data
labelss = np.random.randint(0, num_classes, size=(7530,))
labelss

In [ ]:
from sklearn.model_selection import train_test_split

# Define your STFT data
X = spectrograms

# Define your labels
y = labelss

# Split your data into training, validation, and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [ ]:
np.shape(X_train)

In [ ]:
#new
indices = np.random.permutation(dataall.shape[0])

# Use the shuffled indices to shuffle both data and labels
shuffled_data = dataall[indices]
shuffled_labels = labels[indices]

In [ ]:
import numpy as np
spec_data= spectrograms[:,:] 
# assume your spectrogram data is stored in variable called 'spec_data'
num_samples = spec_data.shape[0]  # number of samples
num_channels = spec_data.shape[1]  # number of channels
num_freq_bins = spec_data.shape[2]  # number of frequency bins
num_time_windows = spec_data.shape[3]  # number of time windows

# create labels for your data (replace with your own labels)
labels = np.random.randint(low=0, high=5, size=num_samples)

# split data into training, validation, and testing sets (60-20-20 split)
train_split = 0.6
val_split = 0.2

# calculate number of samples for each set
num_train_samples = int(train_split * num_samples)
num_val_samples = int(val_split * num_samples)
num_test_samples = num_samples - num_train_samples - num_val_samples

# shuffle indices of samples
shuffled_indices = np.random.permutation(num_samples)

# split indices into training, validation, and testing sets
train_indices = shuffled_indices[:num_train_samples]
val_indices = shuffled_indices[num_train_samples:num_train_samples+num_val_samples]
test_indices = shuffled_indices[num_train_samples+num_val_samples:]

# create empty arrays to store the split data and labels
train_data = np.zeros((num_train_samples, num_channels, num_freq_bins, num_time_windows))
val_data = np.zeros((num_val_samples, num_channels, num_freq_bins, num_time_windows))
test_data = np.zeros((num_test_samples, num_channels, num_freq_bins, num_time_windows))
train_labels = np.zeros(num_train_samples)
val_labels = np.zeros(num_val_samples)
test_labels = np.zeros(num_test_samples)

# copy data and labels into the split arrays
train_data[:] = spec_data[train_indices]
val_data[:] = spec_data[val_indices]
test_data[:] = spec_data[test_indices]
train_labels[:] = labels[train_indices]
val_labels[:] = labels[val_indices]
test_labels[:] = labels[test_indices]


In [ ]:
np.shape(train_data)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import numpy as np



# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
val_labels_encoded = label_encoder.transform(val_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Convert the labels to categorical format
train_labels_categorical = to_categorical(train_labels_encoded)
val_labels_categorical = to_categorical(val_labels_encoded)
test_labels_categorical = to_categorical(test_labels_encoded)

# Reshape the data for 1D CNN
train_data_1d = train_data.reshape(train_data.shape[0], train_data.shape[1]*train_data.shape[2])
val_data_1d = val_data.reshape(val_data.shape[0], val_data.shape[1]*val_data.shape[2])
test_data_1d = test_data.reshape(test_data.shape[0], test_data.shape[1]*test_data.shape[2])

# Build the 1D CNN model
model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=(train_data.shape[1], train_data.shape[2])))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(train_labels_categorical.shape[1], activation='softmax'))

# Compile the model



In [ ]:
np.shape(train_data_1d)

In [ ]:
np.shape(train_labels_categorical,)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=10, batch_size=32)

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_data, test_labels_categorical)
print("Test loss:", test_loss)
print("Test accuracy:", test_acc)




In [ ]:
import numpy as np

# Assuming your input data is stored in variables x_train, y_train, x_val, y_val, x_test, y_test

# Reshape the data to have the expected shape
x_train = np.reshape(train_data, (train_data.shape[0], 129, 36))
x_val = np.reshape(val_data, (val_data.shape[0], 129, 36))
x_test = np.reshape(test_data, (test_data.shape[0], 129, 36))

# Create the sequential model and add layers
model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(129, 36)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(5, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=32)

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(129, 9)),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# train the model
history = model.fit(train_data, train_labels, epochs=10, validation_data=(val_data, val_labels))

# evaluate the model on test set
test_loss, test_acc = model.evaluate(test_data, test_labels)
print('Test accuracy:', test_acc)

In [ ]:
indices = np.random.permutation(dataall.shape[0])

# Use the shuffled indices to shuffle both data and labels
shuffled_data = dataall[indices]
shuffled_labels = labels[indices]

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

def split_data_equally(X, y, test_size=0.4, random_state=None):
    # Get unique labels in y and their indices
    labels, indices = np.unique(y, return_index=True)
    n_labels = len(labels)
    
    # Split each label's indices into test and train
    train_indices = []
    test_indices = []
    for i in range(n_labels):
        label_indices = np.where(y == labels[i])[0]
        label_train_indices, label_test_indices = train_test_split(label_indices, test_size=test_size, 
                                                                    random_state=random_state)
        train_indices += list(label_train_indices)
        test_indices += list(label_test_indices)
    
    # Shuffle train and test indices
    np.random.shuffle(train_indices)
    np.random.shuffle(test_indices)
    
    # Get corresponding data and labels
    X_train, y_train = X[train_indices], y[train_indices]
    X_test, y_test = X[test_indices], y[test_indices]
    
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = split_data_equally(shuffled_data,shuffled_labels,test_size=0.3 )